# PHS564 — Lecture 09 (Student)
## Standardization + (parametric) g-formula (and “why model”) (MIMIC-IV Demo)

### Learning goals
- Derive and implement standardization:
- \(Pr(Y^a=1)=\sum_l Pr(Y=1|A=a,L=l)\,Pr(L=l)\) (discrete \(L\))
- parametric g-formula for high-dimensional \(L\)
- Connect **estimand → identification → estimator → computation** (this is the “why model” message).
- Compare g-formula vs IPW and articulate tradeoffs (model dependence vs weight instability).
- Bootstrap the full procedure for uncertainty.

### Required reading
- Hernán & Robins, g-formula/standardization sections (aligned chapter in *What If*).

**Rules for this notebook**
- Only edit cells marked **TODO**.
- Do not change the overall structure/cell order.


In [ ]:
# Colab bootstrap (run this first if you opened from a Colab badge)
# - Clones the repo into /content/PHS564 (if needed)
# - Installs requirements
# - Adds repo to sys.path

from __future__ import annotations

import os
import sys
import subprocess
from pathlib import Path


def _in_colab() -> bool:
    return "google.colab" in sys.modules


if _in_colab():
    REPO_URL = "https://github.com/vafaei-ar/PHS564.git"
    TARGET_DIR = Path("/content/PHS564")

    if not (TARGET_DIR / "requirements.txt").exists():
        print("Cloning course repo into Colab runtime...")
        subprocess.run(["git", "clone", "--depth", "1", REPO_URL, str(TARGET_DIR)], check=True)

    os.chdir(TARGET_DIR)

    print("Installing requirements...")
    subprocess.run([sys.executable, "-m", "pip", "-q", "install", "-r", "requirements.txt"], check=True)

    if str(TARGET_DIR) not in sys.path:
        sys.path.insert(0, str(TARGET_DIR))

    print("✓ Colab setup complete. Now run the rest of the notebook.")
else:
    print("Not running in Colab; skipping Colab bootstrap.")


### Setup

This notebook is designed to run **locally** or in **Google Colab**.

**If you opened from the Colab badge (recommended):**
1) Run the first code cell titled **“Colab bootstrap”** (it clones the repo + installs requirements)
2) Run the notebook top-to-bottom.

**If you are running locally:**
- Install dependencies from `requirements.txt` (see the repo `README.md`), then run top-to-bottom.


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
RNG = np.random.default_rng(564)

# Locate repo root (works when running from lectures/Lxx.../student or /instructor)
THIS_DIR = Path.cwd()
REPO_ROOT = THIS_DIR
for _ in range(4):
    if (REPO_ROOT / "requirements.txt").exists() or (REPO_ROOT / "README.md").exists():
        break
    REPO_ROOT = REPO_ROOT.parent

DATA_DIR = REPO_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"

print("Working directory:", THIS_DIR)
print("Repo root:", REPO_ROOT)
print("Processed data dir exists:", PROC_DIR.exists())


### Build the processed cohort extract (required)

This lecture expects an analysis-ready cohort file in `data/processed/`.

If it’s missing, run the next cell to:
1) download MIMIC-IV Demo into `data/raw/` (if needed)
2) build the processed cohort extracts into `data/processed/`

**Exposure mode (A):** default uses `admission_type` (contains “EMER”). You can switch to `vitals` by changing `EXPOSURE_MODE` in the code cell.

In [ ]:
# Build processed cohort extracts from raw MIMIC-IV Demo (safe to re-run)
# This will create cohort files under data/processed/.

EXPOSURE_MODE = "admission_type"  # or "vitals"
HR_THRESHOLD = 100.0  # used only if EXPOSURE_MODE == "vitals"

try:
    from data.download_data import download_mimic_demo
    from data.build_processed_extracts_demo import build_processed_extracts

    mimic_dir = RAW_DIR / "mimic-iv-demo-2.2"
    if not mimic_dir.exists():
        print("Downloading raw MIMIC-IV Demo (v2.2) to data/raw/ ...")
        download_mimic_demo(out_dir=RAW_DIR, version="2.2", method="python")
    else:
        print("✓ Raw MIMIC-IV Demo already present.")

    out_paths = build_processed_extracts(exposure_mode=EXPOSURE_MODE, hr_threshold=HR_THRESHOLD)
    print("✓ Built processed cohorts:")
    for k, v in out_paths.items():
        print(f"  {k}: {v}")
except Exception as e:
    print("Could not build processed cohort extracts in this environment.")
    print("Error:", e)
    print("If you already have the cohort file, place it in data/processed/ and re-run.")

### Optional: download raw MIMIC-IV Demo tables

Not required for the homework pipeline. Skip unless your instructor asks you to explore the raw Demo tables in `data/raw/`.

In [ ]:
# Optional: explore raw tables (safe to skip)
# The cohort-build cell above already downloads what you need for this lecture.

try:
    from data.download_data import download_mimic_demo

    download_mimic_demo(out_dir=RAW_DIR, version="2.2", method="python")
except Exception as e:
    print("Skipping raw MIMIC-IV Demo download.")
    print("Error:", e)

## Data
This lecture uses a **processed cohort extract** derived from MIMIC-IV Demo.

Expected file: `data/processed/cohort_L09_gformula.parquet` (or `.csv`).

If this file is missing:
- Run the “Build the processed cohort extract” cell above, or
- Run locally: `python data/build_processed_extracts_demo.py --exposure-mode admission_type`


### Column definitions (this cohort)

These are defined by the cohort builder (`data/build_processed_extracts_demo.py`):

- `A` (treatment):
  - `EXPOSURE_MODE = "admission_type"` (default): `A = 1` if `admission_type` contains `"EMER"` (e.g., "EW EMER.", "DIRECT EMER.")
  - `EXPOSURE_MODE = "vitals"` (optional): `A = 1` if mean heart rate in the **first 6 hours** of the ICU stay is `> HR_THRESHOLD`
- `Y` (outcome): `Y = 1` if the patient **died in-hospital** (`hospital_expire_flag`)
- `A_label`: human-readable description of the `A` definition


In [ ]:
# Statsmodels for regression (logit/ols); installed via requirements.txt
import statsmodels.api as sm
import statsmodels.formula.api as smf

parquet_path = PROC_DIR / "cohort_L09_gformula.parquet"
csv_path = PROC_DIR / "cohort_L09_gformula.csv"

if parquet_path.exists():
    df = pd.read_parquet(parquet_path)
elif csv_path.exists():
    df = pd.read_csv(csv_path)
else:
    raise FileNotFoundError(
        "Missing cohort file for L09. Build it via:\n"
        "  - Colab: run the build cell above\n"
        "  - Local: python data/build_processed_extracts_demo.py --exposure-mode admission_type\n"
    )

df.head()

### Choose variables


In [ ]:
# Variables (match the built cohort)
A = "A"   # treatment column
Y = "Y"   # outcome (0/1): in-hospital death

# Create simple numeric covariates for modeling
if "sex_male" not in df.columns:
    df["sex_male"] = (df["sex"].astype(str).str.upper() == "M").astype(int)

# Baseline covariates/confounders
L_list = ["age", "sex_male", "los"]

analysis_vars = [A, Y] + L_list
d = df[analysis_vars].dropna().copy()
d.head()

## Part A — Outcome regression model
Students only specify the model.


In [ ]:
# TODO: specify model for E[Y | A, L]
# If Y is binary, use logit; if continuous, use OLS.
# Start with: "Y ~ A + age + sex"
outcome_formula = Y + " ~ " + " + ".join([A] + L_list)

# Choose model type (auto-guess binary if only 0/1 values)
is_binary = set(d[Y].dropna().unique()).issubset({0,1})

if is_binary:
    y_model = smf.logit(outcome_formula, data=d).fit(disp=False)
else:
    y_model = smf.ols(outcome_formula, data=d).fit()

y_model.summary().tables[0]

## Part B — Standardization (parametric g-formula)
Predict outcomes under A=1 and A=0 for everyone, then average.


In [ ]:
d1 = d.copy(); d1[A] = 1
d0 = d.copy(); d0[A] = 0

pred1 = y_model.predict(d1)
pred0 = y_model.predict(d0)

mu1 = float(np.mean(pred1))
mu0 = float(np.mean(pred0))
ate = mu1 - mu0
{"mu1": mu1, "mu0": mu0, "ATE": ate}

### TODO B1 — Bootstrap CI for ATE (optional)


In [ ]:
def gformula_ate(data: pd.DataFrame) -> float:
    # Fit model and compute ATE (same specification as above)
    is_binary = set(data[Y].dropna().unique()).issubset({0,1})
    if is_binary:
        m = smf.logit(outcome_formula, data=data).fit(disp=False)
    else:
        m = smf.ols(outcome_formula, data=data).fit()
    d1 = data.copy(); d1[A]=1
    d0 = data.copy(); d0[A]=0
    return float(m.predict(d1).mean() - m.predict(d0).mean())

def bootstrap_ci(data: pd.DataFrame, B: int = 300) -> tuple[float,float,float]:
    vals=[]
    n=len(data)
    for _ in range(B):
        samp=data.sample(n=n, replace=True)
        vals.append(gformula_ate(samp))
    vals=np.array(vals)
    return float(vals.mean()), float(np.quantile(vals,0.025)), float(np.quantile(vals,0.975))

bootstrap_ci(d, B=200)

## Reflection
1) What assumptions does parametric g-formula add beyond identification assumptions?
2) How would model misspecification show up?
